In [1]:
import datasets
import polars as pl
from datetime import datetime


The datasets been generated using this [fork](https://github.com/baberabb/cce-python). The main logic is
for each registration and each matching renewal entry:

    1. check if date matches -> exact match
        2. if not then check if year matches
            3. if not then check if the normalized authors/title match


In [260]:
renewals_unmatched = datasets.load_dataset("baber/pdbooks", "renewals_unmatched")[
    "train"
].to_pandas()
renewals_matched = datasets.load_dataset("baber/pdbooks", "renewals_matched")[
    "train"
].to_pandas()
registrations_all = datasets.load_dataset("baber/pdbooks", "registrations_all")[
    "train"
].to_pandas()
registrations_unmatched = datasets.load_dataset(
    "baber/pdbooks", "registrations_not_renewed"
)["train"].to_pandas()

We'll use polars, but it's interchangeable with pandas with .to_pandas()

In [384]:
ren_unmatched = pl.from_pandas(renewals_unmatched)
ren_matched = pl.from_pandas(renewals_matched)
reg_all = pl.from_pandas(registrations_all)
reg_unmatched = pl.from_pandas(registrations_unmatched)

# print(f"Total renewals unmatched: {len(ren_unmatched)}")
print(f"Total registrations remaining: {len(reg_unmatched)}")

Total registrations remaining: 560350


These are alot of unmatched renewals but these include non-book entries as well. Sub-setting just the registration numbers we have in the reg dataset and filtering the dates we get ~8000 matched numbers with _all_ registrations and ~4000 with the registrations remaining (unmatched, not foreign, etc.). Looking at all registrations:

In [385]:
def filter_dates(x):
    return pl.col(x).list.eval(pl.element().str.to_date(strict=False, exact=False).is_between(datetime(1929, 1, 1), datetime(1964, 1, 1))).list.any() | pl.col(x).is_null() | pl.col(x).list.eval(pl.element().len() < 1).list.any()

def normalize_regnums(x):
    return pl.col(x).str.replace_all("-", "")

In [398]:
from polars import col

reg_all_ = (
    reg_all.lazy()
    .with_columns(pl.col(pl.Utf8).replace("", None))
    .filter(filter_dates("reg_dates"))
    .explode("regnums")
    .with_columns(normalize_regnums("regnums"))
)

ren_unmatched_ = (
    ren_unmatched.lazy()
    .with_columns(pl.col(pl.Utf8).replace("", None))
    .with_columns(
        # when title contains by and author is null then we'll just use title
        prompt_ren=pl.when(col("title").is_null())
        .then(col("full_text").str.split("©").list.first())
        .when(col("author").is_null() & col("title").str.contains("by"))
        .then(pl.format("{}", col("title")))
        # if both author and title then use "{title} by {author}"
        .when(col("author").is_not_null())
        .then(pl.format("{} by {}", col("title"), col("author")))
        # else by UNKNOWN
        .otherwise(pl.format("{} by {}", col("title"), pl.lit("UNKNOWN")))
    )
    .explode("regnum")
    .with_columns(normalize_regnums("regnum"))
    .join(
        ren_matched.lazy().select("uuid"),
        how="anti",
        on="uuid",
    )
    .filter(filter_dates("reg_date"))
    .rename({"uuid": "uuid_ren"})
)

In [404]:
final_df = (
    reg_all_.join(
        ren_unmatched_.lazy(), how="inner", left_on="regnums", right_on="regnum"
    )
    .with_columns(
        # this logic leaves out the case where there's only authors and publishers
        # 43 rows however none of them are a match
        prompt_reg=pl.when(col("authors").is_not_null())
        .then(
            pl.format(
                "{} by {}",
                col("title"),
                col("authors").list.join(" & "),
            )
        )
        .when(col("authors").is_null() & col("publishers").is_not_null())
        .then(
            pl.format(
                "{} published by {}", col("title"), col("publishers").list.join(" & ")
            )
        )
        .otherwise(pl.format("{} by UNKNOWN", pl.col("title")))
    )
    .unique(["prompt_reg", "prompt_ren"])
    .filter(pl.col("prompt_reg").is_not_null())
)

In [410]:
final_df.collect()

uuid,regnums,reg_dates,title,authors,publishers,disposition,year,group_title,group_uuid,notes,parent,new_matter_claimed,previous_publications,uuid_ren,reg_date,renewal_id,renewal_date,author,title_right,new_matter,see_also_renewal,see_also_registration,full_text,claimants,notes_right,prompt_ren,prompt_reg
str,str,list[str],str,list[str],list[str],str,str,str,str,list[str],str,list[str],list[str],str,list[str],str,str,str,str,str,list[null],list[null],str,str,str,str,str
"""FC1116C3-7454-…","""A181722""","[""1944-05-23""]","""Strangers in m…","[""Buirgy (Mary)""]",null,"""Not renewed.""","""1944""",null,null,null,null,null,null,"""feedd024-b95d-…","[""1955-03-16""]","""RE173961""",null,null,"""Teacher's manu…",null,[],[],null,"""Holt, Rinehart…",null,"""Teacher's manu…","""Strangers in m…"
"""063B6B35-7455-…","""A184743""","[""1944-11-30""]","""Year book of t…",null,"[""Year book publishers, inc.""]","""Not renewed.""","""1944""",null,null,null,null,null,null,"""c9fca8e4-8a05-…","[""1954-09-23""]","""RE150894""",null,null,"""Smoke jumpers.…",null,[],[],null,"""Nels Jorgensen…",null,"""Smoke jumpers.…","""Year book of t…"
"""3D6C9285-734A-…","""A176738""","[""1943-10-13""]","""Turquoise path…","[""Rousseau (Christine McConnell)""]","[""Broadman press""]","""Not renewed.""","""1943""",null,null,null,null,null,null,"""d0c17431-56e1-…","[""1955-02-23""]","""RE159740""",null,null,"""History of art…",null,[],[],null,"""Jean Anne Vinc…",null,"""History of art…","""Turquoise path…"
"""428DAE0A-734A-…","""A170646""","[""1943-01-26""]","""American agric…","[""Barger (Harold)"", ""H. Barger"", ""Hans H. Landsberg""]","[""Natl. bureau of economic research, inc.""]","""Not renewed.""","""1943""",null,null,null,null,null,null,"""3586e70a-a649-…","[""1950-12-23""]","""RE7313""",null,null,"""For presenting…",null,[],[],null,"""Helen Seitter …",null,"""For presenting…","""American agric…"
"""429007E2-734A-…","""A170652""","[""1943-01-18""]","""L’imitation de…","[""Lelen (J. M.)""]","[""Catholic book pub. co.""]","""Not renewed.""","""1943""",null,null,null,null,null,null,"""b08f6330-ef8f-…","[""1950-12-20""]","""RE7319""",null,null,"""Voices from th…",null,[],[],null,"""Helen Seitter …",null,"""Voices from th…","""L’imitation de…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""5B4C5FCB-6E24-…","""A5865""",[],"""For all mankin…","[""Blum, Léon"", ""Léon Blum"", ""W. Pickles""]",null,null,"""1946""",null,null,null,null,null,null,"""e9d8c94e-c6db-…","[""1929-05-04""]","""R162097""","""1956-01-04""","""MILLARD, WILLI…","""The supplement…",null,[],[],"""MILLARD, WILLI…","""William Barret…",null,"""The supplement…","""For all mankin…"
"""620E6087-6E24-…","""AF1182""",[],"""… Histoire gén…",null,null,null,"""1946""",null,null,"[""illus. (1 mounted col.; facsims., maps) plates (part mounted col.) 30×25cm. At head of title: Sous la directio. de mm. Maxims Gorce et Raoul Mortier. Contents.--[v. 1] Grèce. Rome.--[v. 2] Indo-Iraníens. Judaisme. Origines chrétianismes. Christianismes orientaux""]",null,"[""v. 1""]",null,"""2cdea580-668f-…","[""1944-06-30""]","""R511013""","""1971-08-31""","""MORTIER, RAOUL…","""Histoire gener…",null,[],[],"""MORTIER, RAOUL…","""Mme Mortier, n…",null,"""Histoire gener…","""… Histoire gén…"
"""632182AF-6E24-…","""A5117""",[],"""The wonderworl…","[""Knox, Warren William"", ""Warren Knox"", … ""Frank Hubbard""]","[""Charles Scribner's sons""]",null,"""1946""",null,null,"[""Book 4 [5, 9]"", ""Book 9 by Morris Meistor, Ralph E. Keirstead and Lois M. Shoemaker""]",null,"[""revisions, book 4, 5""]",null,"""3e806bfa-24fe-…","[""1945-12-03""]","""R564543""","""1973-12-03""","""Alice Ormond C…","""With bated bre…",null,[],[],"""R564543. With …","""Chita Ormond C…",null,"""With bated bre…","""The wonderworl…"


In [408]:
final_df.collect().write_parquet(
    "/Users/baber/PycharmProjects/cce-python/llm/test_matching/renewals_unmatched_for_llm.parquet"
)

Get the matched UUIDS. We have matched 518 renewals and 185 registration uuids. The registrations are alot less because most of them were "child" entries with a reg number but no uuid.
Matched 517 unique regnumbers.

In [412]:
import json

with open(
    "/Users/baber/PycharmProjects/cce-python/output/FINAL-registration_matches_from_llm.ndjson",
) as f:
    matches = []
    for entry in f:
        matches.append(json.loads(entry))

In [425]:
pl.from_dicts(matches).lazy().join(
    final_df,
    how="inner",
    left_on=["reg_uuid", "ren_uuid"],
    right_on=["uuid", "uuid_ren"],
    join_nulls=True,
).collect().unique("regnums")

reg_uuid,ren_uuid,regnums,reg_dates,title,authors,publishers,disposition,year,group_title,group_uuid,notes,parent,new_matter_claimed,previous_publications,reg_date,renewal_id,renewal_date,author,title_right,new_matter,see_also_renewal,see_also_registration,full_text,claimants,notes_right,prompt_ren,prompt_reg
str,str,str,list[str],str,list[str],list[str],str,str,str,str,list[str],str,list[str],list[str],list[str],str,str,str,str,str,list[null],list[null],str,str,str,str,str
"""28D8EBAE-7359-…","""6da685b8-be80-…","""A151239""","[""1941-03-04""]","""Pennsylvania G…","[""Pennsylvania German folklore soc""]",null,"""Not renewed.""","""1941""",null,null,null,null,null,null,"[""1951-03-04""]","""R433307""","""1968-04-09""","""ZIEGLER, SAMUE…","""The Pennsylvan…",null,[],[],"""ZIEGLER, SAMUE…","""Pennsylvania G…",null,"""The Pennsylvan…","""Pennsylvania G…"
"""C7219039-7893-…","""d8670be3-dfa7-…","""A30335""","[""1948-11-21""]","""Modem wonders …","[""LEYSON, BURR WATKINS.""]","[""E. P. Dutton & Co., inc. New York""]","""Not renewed.""","""1949""",null,null,null,null,null,null,"[""1949-02-21""]","""R657956""","""1977-02-14""","""Burr W. Leyson…","""Modern wonders…",null,[],[],"""R657956. Moder…","""E. P. Dutton|P…",null,"""Modern wonders…","""Modem wonders …"
"""DB455811-728D-…","""468a7bb0-5e31-…","""A215427""","[""1955-12-01""]","""Pt.6-10.""",null,null,"""Not renewed.""","""1955""",null,null,null,null,null,null,"[""1952-06-06""]","""RE74054""",null,null,"""Immortal poems…",null,[],[],null,"""Gulf & Western…",null,"""Immortal poems…","""Pt.6-10. by UN…"
"""18E952B8-6DC2-…","""35aa5636-148d-…","""A614541""","[""1962-02-20""]","""Western campsi…","[""SUNSET."", ""the editorial staff of Sunset books."", ""SUNSET.""]","[""Lane Book Co."", ""Lane Book Co.""]","""Not renewed.""","""1963""",null,"""5066e611778449…",null,null,null,null,"[""1963-02-20""]","""RE542034""",null,null,"""Western campsi…",null,[],[],null,"""Sunset Publish…",null,"""Western campsi…","""Western campsi…"
null,"""ed9f857c-a0d3-…","""A106774""","[""1953-01-24"", ""1"", … ""7""]","""SUPERMAN.""","[""Wayne Boring.""]","[""National Comics Publications, inc.""]","""Not renewed.""",null,null,null,null,"""CD1CC570-6F17-…",null,null,"[""1951-01-24""]","""RE33061""",null,null,"""Superman. Rel…",null,[],[],null,"""Comics, Inc.|P…",null,"""Superman. Rel…","""SUPERMAN. by W…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""BA260FBC-6C64-…","""c8178def-877b-…","""A69503""","[""1934-01-25""]","""Modern America…","[""Cal. R. Fisk""]","[""Modern American corporation"", ""Modern Amer. corporation.""]","""Not renewed.""","""1934""",null,null,"[""Paged continuously. Based on the New student’s reference work. Vol. 8 contains Lesson outlines and classified questions in nature-work, geography, history, biography, literature, the arts and sciences, industries and inventions, with page references.""]",null,null,null,"[""1952-08-13""]","""RE57093""",null,null,"""The New Funk &…",null,[],[],null,"""Unicorn Press,…",null,"""The New Funk &…","""Modern America…"
null,"""db282bd1-3c8d-…","""A106740""","[""1953-05-17"", ""1"", … ""7""]","""SUPERMAN.""","[""Wayne Boring.""]","[""National Comics Publications, inc.""]","""Not renewed.""",null,null,null,null,"""CD1CC570-6F17-…",null,null,"[""1950-05-17""]","""RE5605""",null,null,"""Superman. Rel…",null,[],[],null,"""Comics, Inc.|P…",null,"""Superman. Rel…","""SUPERMAN. by W…"
null,"""4aa9b3f1-bfd2-…","""A266812""","[""1957-07-03"", ""1"", … ""9""]","""Donald Duck.""","[""DISNEY (WALT) PRODUCTIONS.""]","[""Walt Disney Productions.""]","""Not renewed.""",null,null,null,"[""16-21Jul56""]","""293EB567-72C4-…",null,null,"[""1956-07-03""]","""RE199736""",null,null,"""Donald Duck. …",null,[],[],null,"""Walt Disney Pr…",null,"""Donald Duck. …","""Donald Duck. b…"
